# BACKTESTING VALIDACION

## TEORIA

### LOGICA (IDEA)

Sistema de Larry Connors RSI2

* Activo:
  * SPX

* Long:
  * Entrada
    * Rsi2 < 10 & Precio > Sma 200
  * Salida
    * Rsi2 > 50 | Precio < Sma 200

### SISTEMA

* Entrada -> data IS
* Evitar sesgo de anticipacion
* Nunca mezclar long y short en el mismo sistema -> mejor dos sistemas
* Buscamos:
  * Sistemas que funcionen bien
  * Sistemas que funcionen mal
* Salida -> Señales y Position Size

* Es el caso mas complejo (optimizado):
  * Entrada y salida por indicador
  * Y que no este dentro de la posicion mas de un numero de velas
  * Y que salga por take profit o stop loss
* Caso mas simple:
  * Entrada por indicador
  * Salida por ...
  * Habria que programarlo...
* CONTROL ABSOLUTO DE LAS SEÑALES:
  * Tiempo
  * Pnl
* Solo Importa:
  * data.TRADE
  * data.ROID
  * data.ROIACUM

### BACKTESTING RESULTADO

* Entrada
  * Señales y Position Size
  * Resultados del sistema

### BACKTESTING OPTIMIZACION

* Encontrar los parametros que hacen mas robusto al sistema
* Sistemas con poco parametros son los mejores
* Si el sistema tiene mucho parámetros lo podemos torturar hasta que nos diga lo que queremos
* La data debe comenzar en el mismo punto, el calculo de indicadores puede afectar
* Nuestro sistema:
  * Sma -> 200
  * Rsi -> 2, 10, 50

## IMPORTACIONES

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

import matplotlib.pyplot as plt

from IPython.display import HTML, display

import warnings
warnings.filterwarnings('ignore')

## DATA IS

In [ ]:
data = pd.read_excel('dfIS.xlsx', index_col=0)
data

## SETUP

In [ ]:
perSma = 200
perRsi = 2

rsiIn  = 10
rsiOut = 50

salidaVelas = 0

sentido = 'long'

pSize = 1

tp = 0
sl = 0

# se aplica en la entrada para la entrada y la salida
# 0.01 es 0.01% se aplica a la entrada; $10000 -> $1
# 0.02 es 0.02% se aplica a la entrada; $10000 -> $2
comision = 0 
slippage = 0 

## SISTEMA

In [ ]:
def ocpSma(df, periodo = 20, borraNan = False, col = 'Close'):
    
    ''' return data'''
    
    df[f's{periodo}'] = df[col].rolling(periodo).mean()
    
    if borraNan: df.dropna(inplace = True)
        
    return df
    
def ocpRsi(df, periodo = 14, borraNan = False, col = 'Close'):
    
    ''' return data'''
    
    df['dif'] = df[col].diff()

    df['win']     = np.where(df['dif'] > 0, df['dif'], 0)
    df['loss']    = np.where(df['dif'] < 0, abs(df['dif']), 0)
    df['emaWin']  = df.win.ewm(span = periodo).mean()
    df['emaLoss'] = df.loss.ewm(span = periodo).mean()
    df['rs']      = df.emaWin / df.emaLoss

    df[f'rsi{periodo}'] = 100 - (100 / (1+df.rs))

    df.drop(['dif', 'win', 'loss', 'emaWin', 'emaLoss', 'rs'], axis = 1, inplace = True)

    if borraNan: df.dropna(inplace = True)

    return df



# P  -> Posicion
# cP -> cierre Posicion

def dameSistema(df, perSma, perRsi, rsiIn, rsiOut):
    
    df['signal'] = ''
    
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    
    #df['signal'] = np.where((df[f'rsi{perRsi}'] < rsiIn) , 'P' , df.signal)
    #df['signal'] = np.where((df[f'rsi{perRsi}'] > rsiOut), 'cP', df.signal)
    #data['signal'] = np.where(data.signal.shift(5) == 'P' ,'cP', df.signal)

    df['signal'] = np.where((df[f'rsi{perRsi}'] < rsiIn) & (df.Close > df[f's{perSma}']) , 'P' , df.signal)
    #df['signal'] = np.where((df[f'rsi{perRsi}'] > rsiOut) , 'cP', data.signal)
    df['signal'] = np.where((df[f'rsi{perRsi}'] > rsiOut) | (df.Close < df[f's{perSma}']) , 'cP', df.signal)
    
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>  
    # evitar sesgo de anticipacion
    df['position'] = ''
    df['position'] = df.signal.shift()

    return df

def damePosition(df):

    df['tradeInd'] = ''
    
    inTrade = False
    for i in df.index:
        
        cell = df.loc[i, 'position']
    
        if pd.isna(cell):
            pass # Sin señal, no cambiamos estado
        else:
            if cell == 'P':
                if not inTrade:
                    inTrade = True
                    df.loc[i, 'tradeInd'] = 'In'  # Entrada
                else: # Ya en posicion
                    df.loc[i, 'tradeInd'] = 'p'
            elif cell == 'cP':
                if inTrade:
                    df.loc[i, 'tradeInd'] = 'Out'  # Salida
                inTrade = False
            else:
                if inTrade:
                    df.loc[i, 'tradeInd'] = 'p'
    
    df['TRADE'] = df.tradeInd

    return df


def dameSalidaVelas(df, num):

    '''
    Si num vale 0 no se tiene en cuenta la salida por velas
    '''

    df['velas'] = 0
    df = df.reset_index(drop=False)
    for i in df.index:
    
        if i > 1:
        
            cell = df.loc[i, 'TRADE']
            
            if   cell == 'In' : df.loc[i, 'velas'] = 1
            elif cell == 'p'  : df.loc[i, 'velas'] = df.loc[i-1, 'velas'] +1
                
    numMaxVelas = num
    if numMaxVelas:
        
        df['tradeVela'] = ''
        
        
        for i in df.index:
        
            cell = df.loc[i, 'TRADE']
            vela = df.loc[i, 'velas']
            
            if vela <= numMaxVelas: df.loc[i, 'tradeVela'] = cell if cell in ['In', 'p'] else ''
        
        df['tradeVela'] = np.where((df['tradeVela'] == '') & (df['tradeVela'].shift().isin(['In', 'p'])),'Out',df['tradeVela'])
        
        df['TRADE'] = df.tradeVela
    
    else:
    
        df['tradeVela'] = df.TRADE

    df.set_index('Date', inplace=True)
    
    return df
    
    
def dameSalidaPnlOLD(df, sentido, tp, sl):

    '''
    tp valores tipo 3.5
    sl valores tipo -1.5

    si alguno vale 0 no se tiene en cuenta ese parametro
    '''

    df['ROID'] = 0
    df = df.reset_index(drop=False)
    for i in df.index:
        
        estado = df.loc[i, 'TRADE']
       
        if i > 0:
            # sentido long
            if   estado == 'In'   : df.loc[i, 'ROID'] = (df.loc[i, 'Close'] / df.loc[i, 'Open'] -1) * 100
            elif estado == 'p'    : df.loc[i, 'ROID'] = (df.loc[i, 'Close'] / df.loc[i-1, 'Close'] - 1) * 100
            elif estado == 'Out'  : df.loc[i, 'ROID'] = (df.loc[i, 'Open']  / df.loc[i-1, 'Close']  - 1) * 100
            else                  : df.loc[i, 'ROID'] = 0
    
    if sentido == 'short': df['ROID'] = - df.ROID

    df['ROIACUM'] = 0
    for i in df.index:
        estado = df.loc[i, 'TRADE']
        roiD   = df.loc[i, 'ROID']
    
        if estado == 'In'          : df.loc[i, 'ROIACUM'] = roiD
        elif estado in ['p', 'Out']: df.loc[i, 'ROIACUM'] = ((1+roiD/100) * (1+df.loc[i-1, 'ROIACUM']/100) -1)*100
    

    df['tradePnl'] = ''
    for i in df.index:
    
        cell    = df.loc[i, 'TRADE']
        roiAcum = df.loc[i, 'ROIACUM']
        
        if   cell == 'In': df.loc[i, 'tradePnl'] = 'In'
        elif cell == 'p':
            #if ((tp != 0 and df.loc[i-1, 'ROIACUM'] > tp) or (sl != 0 and df.loc[i-1, 'ROIACUM'] < sl)): df.loc[i, 'tradePnl'] = 'Out'
            if ((tp != 0 and df.loc[i, 'ROIACUM'] > tp) or (sl != 0 and df.loc[i, 'ROIACUM'] < sl)): df.loc[i, 'tradePnl'] = 'Out'
            else                                                                                       : df.loc[i, 'tradePnl'] = 'p'
        elif cell == 'Out': df.loc[i, 'tradePnl'] = 'Out'

    df['tradePnl'] = np.where((df['tradePnl'] == 'p')   & (df['tradePnl'].shift().isin(['', 'Out'])),'',df['tradePnl'])
    df['tradePnl'] = np.where((df['tradePnl'] == 'Out') & (df['tradePnl'].shift().isin(['', 'Out'])),'',df['tradePnl'])

    df['TRADE'] = df.tradePnl
    
    df.set_index('Date', inplace=True)
    
    return df


def dameSalidaPnl(df, sentido, tp, sl, comision, slippage):

    '''
    tp valores tipo 3.5
    sl valores tipo -1.5
    
    si alguno vale 0 no se tiene en cuenta ese parametro
    
    se aplica en la entrada para la entrada y la salida
    comision 0.01 es 0.01% se aplica a la entrada; $10000 -> $1
    slippage 0.02 es 0.02% se aplica a la entrada; $10000 -> $2

    '''

    def pnlSalida(Open, High, Low, Close, precioRef, sentido, tp, sl):
        '''
        Determina si se alcanza SL/TP o es sesion normal durante una vela
    
        PARAMETROS:
        - Open, High, Low, Close: Precios OHLC de la vela
        - precioRef: Precio de referencia (entrada de la posicion)
        - sentido: 'long' o 'short'
        - sl: Stop Loss en % (ej: -2.0 para -2%), si 0 no se considera
        - tp: Take Profit en % (ej: 0.8 para +0.8%), si 0 no se considera
       
        LOGICA:
        - Prioridad: SL primero, luego TP. En una sesion se mira primero el SL, y luego el TP
        - Si se supera en apertura, ROI real de apertura
        - Si se supera durante vela, ROI teorico por SL/TP
    
        RETORNA:
        - Tupla (tipoSalida, roiPorcentaje)
        - tipoSalida: sltp, o normal
        - roiPorcentaje: ROI REAL en % respecto a precioRef
        '''

        # Funcion auxiliar para calcular ROI real
        def calcularRoi(precio, precioRef, sentido):
            
            roi = (precio - precioRef) / precioRef * 100
            if sentido == 'short': roi = -roi
                
            return roi
    
        # VERIFICAR SL (PRIORIDAD 1)
        if sl:
            
            if sentido == 'long':
                
                precioSl = precioRef * (1 + sl/100)
                
                # Verificar si se toca SL
                if Open <= precioSl or Low <= precioSl:
                    # Si se toca en apertura, usar ROI de apertura
                    if Open <= precioSl: return ('sltp', calcularRoi(Open, precioRef, sentido))
                    # Se toca durante la vela, usar SL teorico    
                    else: return ('sltp', sl)
            
            # short           
            else:  
                
                precioSl = precioRef * (1 - sl/100)
                
                # Verificar si se toca SL
                if Open >= precioSl or High >= precioSl:
                    # Si se toca en apertura, usar ROI de apertura
                    if Open >= precioSl: return ('sltp', calcularRoi(Open, precioRef, sentido))
                    # Se toca durante la vela, usar SL teorico   
                    else: return ('sltp', sl)
    
        # VERIFICAR TP (PRIORIDAD 2)
        if tp:
            
            if sentido == 'long':
                
                precioTp = precioRef * (1 + tp/100)
                
                # Verificar si se toca TP
                if Open >= precioTp or High >= precioTp:
                    # Si se toca en apertura, usar ROI de apertura
                    if Open >= precioTp: return ('sltp', calcularRoi(Open, precioRef, sentido))
                    # Se toca durante la vela, usar TP teorico
                    else: return ('sltp', tp)
            # short           
            else:  
                
                precioTp = precioRef * (1 - tp/100)
                
                # Verificar si se toca TP
                if Open <= precioTp or Low <= precioTp:
                    # Si se toca en apertura, usar ROI de apertura
                    if Open <= precioTp: return ('sltp', calcularRoi(Open, precioRef, sentido))
                    # Se toca durante la vela, usar TP teórico
                    else: return ('sltp', tp)
    
        # SALIDA NORMAL - calcular ROI al cierre
        return ('normal', calcularRoi(Close, precioRef, sentido))

    
    
    
    df['ROID'] = 0
    df['ROIACUM'] = 0  
    
    df['precioRef']  = 0
    df['tipoSalida'] = ''
    
    df = df.reset_index(drop=False)
    
    if tp or sl:
    
        for i in df.index:
            
            estado = df.loc[i, 'TRADE']
    
            O = df.loc[i, 'Open']
            H = df.loc[i, 'High']
            L = df.loc[i, 'Low']
            C = df.loc[i, 'Close']
           
            if i > 0:
                # sentido long
                if   estado == 'In'   : 
    
                    df.loc[i,'tradePnl'] = 'In'
                    
                    precioRef = df.loc[i, 'Open']
                    salida, roi = pnlSalida(O,H,L,C, precioRef, sentido, tp, sl)

                    # solo lo reflejo en al apertura de la posicoin In
                    coste = (comision + slippage)
                    roi -= coste
    
                    # tanto salida normal comum sl y tp
                    df.loc[i, 'ROID']       = roi
                    df.loc[i, 'ROIACUM']    = roi
                    df.loc[i, 'precioRef']  = precioRef
                    df.loc[i, 'tipoSalida'] = salida
    
                elif estado in ['p','Out']:
    
                    # p y Out solo puede tener antes In o p, si no es huerfana y se elimina
                    if df.loc[i-1, 'tradePnl'] not in ['In','p']: 
                        
                        df.loc[i,'tradePnl'] = ''
                        
                        df.loc[i, 'ROID']       = 0
                        df.loc[i, 'ROIACUM']    = 0
                        df.loc[i, 'precioRef']  = 0
                        df.loc[i, 'tipoSalida'] = ''
    
                    # si antes es un In o p
                    else:
                        # una salida por sltp en la anterior
                        if df.loc[i-1,'tipoSalida'] == 'sltp':
    
                            df.loc[i,'tradePnl'] = 'Out'
                            
                            df.loc[i, 'ROID']    = 0
                            df.loc[i, 'ROIACUM'] = df.loc[i-1, 'ROIACUM']
    
                            df.loc[i, 'precioRef']  = 0
                            df.loc[i, 'tipoSalida'] = ''
                        
                        
                        # anterior es normal
                        else:
                            precioRef = df.loc[i-1, 'precioRef']
                            salida, roiRef = pnlSalida(O,H,L,C, precioRef, sentido, tp, sl)
    
                            # salida normal, no se tocan sl o tp
                            if salida == 'normal':
    
                                df.loc[i,'tradePnl'] = estado
                                
                                if estado == 'p': df.loc[i, 'ROID']       = (df.loc[i, 'Close'] / df.loc[i-1, 'Close'] - 1) * 100
                                if estado == 'Out': df.loc[i, 'ROID']     = (df.loc[i, 'Open'] / df.loc[i-1, 'Close'] - 1) * 100
                                    
                                df.loc[i, 'ROIACUM']    = roiRef
                                df.loc[i, 'precioRef']  = precioRef
                                df.loc[i, 'tipoSalida'] = salida
    
                            # se tocan sl o tp
                            else:
    
                                df.loc[i,'tradePnl'] = 'Out'
    
                                df.loc[i, 'ROID'] = ((1+roiRef/100) / (1+ df.loc[i-1, 'ROIACUM']/100) - 1) * 100
                                df.loc[i, 'ROIACUM'] = roiRef
                                
                                df.loc[i, 'precioRef']  = 0
                                df.loc[i, 'tipoSalida'] = salida
    
                # Cuando esta vacia
                else:
                    
                    df.loc[i,'tradePnl'] = ''
                    
                    df.loc[i, 'ROID']       = 0
                    df.loc[i, 'ROIACUM']    = 0
                    df.loc[i, 'precioRef']  = 0
                    df.loc[i, 'tipoSalida'] = ''
                
    
        df['TRADE'] = df.tradePnl
        
        

    # tanto el sl como el tp son 0, solo queda calcular el ROID y el ROIACUM
    # df.TRADE ya tiene los valores correctos de df.tradeVela
    else:
        # solo lo reflejo en al apertura de la posicoin In
        coste = (comision + slippage)
        for i in df.index:
            
            if i > 0:
                    # sentido long
                    estado = df.loc[i, 'TRADE']
                
                    if   estado == 'In'   :
                        df.loc[i, 'ROID']    = (df.loc[i, 'Close'] / df.loc[i, 'Open'] -1) * 100 - coste
                        df.loc[i, 'ROIACUM'] = df.loc[i, 'ROID']
                        
                    elif estado == 'p'    :
                        df.loc[i, 'ROID']    = (df.loc[i, 'Close'] / df.loc[i-1, 'Close'] - 1) * 100
                        df.loc[i, 'ROIACUM'] = ((1+df.loc[i, 'ROID']/100) * (1+df.loc[i-1, 'ROIACUM']/100) -1)*100
                        
                    elif estado == 'Out'  : 
                        df.loc[i, 'ROID']    = (df.loc[i, 'Open']  / df.loc[i-1, 'Close']  - 1) * 100
                        df.loc[i, 'ROIACUM'] = ((1+df.loc[i, 'ROID']/100) * (1+df.loc[i-1, 'ROIACUM']/100) -1)*100
                        
                    else                  : 
                        df.loc[i, 'ROID']    = 0
                        df.loc[i, 'ROIACUM'] = 0

        if sentido != 'long':
            df[ 'ROID']    = -df[ 'ROID']
            df['ROIACUM']  = -df['ROIACUM']
            

    df.set_index('Date', inplace=True)

    ''' '''

    # PARA COMROBAR QUE EL ROID CALCULADO POR SL O TP TIENE SENTIDO
    # roiComprobar es el roiD calculado a partir de tradeVela

    df['roiComprobar'] = 0

    df = df.reset_index(drop=False)
    for i in df.index:
        
        estado = df.loc[i, 'tradeVela']
      
        if i > 0:
    
            if estado   == 'In'   : df.loc[i, 'roiComprobar'] = (df.loc[i, 'Close'] / df.loc[i, 'Open'] -1) * 100
            elif estado == 'p'    : df.loc[i, 'roiComprobar'] = (df.loc[i, 'Close'] / df.loc[i-1, 'Close'] - 1) * 100
            elif estado == 'Out'  : df.loc[i, 'roiComprobar'] = (df.loc[i, 'Open']  / df.loc[i-1, 'Close']  - 1) * 100
            else                  : df.loc[i, 'roiComprobar'] = 0
    
    df.set_index('Date', inplace=True)
    
    return df

def dameGraficoSistema(df, numVelas, perSma, perRsi, rsiIn, rsiOut):

    dg = df.tail(numVelas).copy()

    fig, axes = plt.subplots(2, 1, sharex=True, figsize=(10, 8), gridspec_kw={'height_ratios': [2, 1]})
    
    # --- Primer subplot Precio ---
    ax0 = axes[0]
    
    ax0.plot(dg.index, dg['Close'], color='black', label='Close')
    ax0.plot(dg.index, dg[f's{perSma}'], color = 'blue', linestyle='--', label = 'Sma200', alpha=0.5)
    
    # Agregar puntos verdes y rojos
    dateInTrade  = dg.index[dg['TRADE'] == 'In']
    dateOutTrade = dg.index[dg['TRADE'] == 'Out']
    ax0.scatter(dateInTrade , dg.loc[dateInTrade, 'Close'], color='green', label='In', marker='o', alpha=0.5)
    ax0.scatter(dateOutTrade, dg.loc[dateOutTrade,'Close'], color='red', label='Out', marker='o', alpha=0.5)
    
    ax0.set_title('GRÁFICO DE PRECIO') 
    ax0.set_ylabel('Close')
    ax0.legend(loc='upper left')
    ax0.grid(True)
    
    # --- Segundo subplot Indicador ---
    ax1 = axes[1]
    
    ax1.plot(dg.index, dg[f'rsi{perRsi}'], color='gray', label='rsi2')
    
    ax1.axhline(90, color='red', linestyle='--', linewidth=0.8)
    ax1.axhline(rsiOut, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
    ax1.axhline(rsiIn , color='green', linestyle='--', linewidth=0.8, alpha=0.5)
    
    ax1.set_title('INDICADORES') 
    ax1.set_ylabel('rsi2')
    ax1.set_xlabel('Date') 
    ax1.legend(loc='upper left')
    #ax1.grid(True) 
    
    # Ajustar el layout para evitar solapamientos entre titulos y ejes
    plt.tight_layout()

    plt.show()

    return